In [18]:
import joblib
import os
import pandas as pd
import numpy as np
import datetime
from tensorflow.keras.models import load_model
import Funtions as fn

In [9]:
data_modal_EMA = joblib.load('data/modal_biaya_harian_EMA.pkl')

In [10]:
meta_model_EMA = joblib.load('model/peramalan_EMA/meta_model.pkl')
dir_EMA = "model/peramalan_EMA"
model_files_EMA = sorted([f for f in os.listdir(dir_EMA) if f.endswith(".h5")])
trained_models_EMA = [load_model(os.path.join(dir_EMA, f)) for f in model_files_EMA]

In [11]:
PREDICTION_DAYS = 7
SEQUENCE_LENGTH = 30

In [12]:
scaler, data_scaled, X_train, y_train, X_val, y_val, train_dates, val_dates = fn.prepare_data(data_modal_EMA, SEQUENCE_LENGTH)

In [13]:
individual_predictions_train, individual_predictions_val = fn.individual_predictions(trained_models_EMA, X_train, X_val)

9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [16]:
train_preds = list(individual_predictions_train.values())
val_preds = list(individual_predictions_val.values())

X_stack_train = np.stack(train_preds, axis=1)
X_stack_val = np.stack(val_preds, axis=1)

In [17]:
ensemble_preds_hist_train = meta_model_EMA.predict(X_stack_train)
ensemble_preds_hist_val = meta_model_EMA.predict(X_stack_val)

In [19]:
last_date = data_modal_EMA['tgl_transaksi'].iloc[-1]
if isinstance(last_date, pd.Timestamp):
    last_date = last_date.date()

future_dates = [last_date + datetime.timedelta(days=i+1) for i in range(PREDICTION_DAYS)]

last_sequence = data_scaled[-SEQUENCE_LENGTH:]

In [20]:
future_predictions = fn.predict_ensemble(trained_models_EMA, scaler, last_sequence, meta_model_EMA)    

In [22]:
df_future_predictions = pd.DataFrame({
    'Tanggal': future_dates,
    'Prediksi Modal': future_predictions.flatten()
})
df_future_predictions.set_index('Tanggal', inplace=True)

df_future_predictions

,Prediksi Modal
Tanggal,
2008-12-31,1.788809e+08
2009-01-01,1.776290e+08
2009-01-02,1.770529e+08
2009-01-03,1.767014e+08
2009-01-04,1.763849e+08
2009-01-05,1.760616e+08
2009-01-06,1.757841e+08


In [23]:
fn.plot_ensemble_predictions(
        data_modal_EMA,
        train_dates,
        val_dates,
        scaler,
        ensemble_preds_hist_train,
        ensemble_preds_hist_val,
        df_future_predictions
    )

In [24]:
fn.plot_future_predictions(future_dates, future_predictions)

In [26]:
mape, mae, mse, rmse  = fn.evaluate_model(y_val, ensemble_preds_hist_val)

Evaluasi hasil prediksi pada Data Validasi:
MAE  : 0.0638
MAPE : 9.7631%
MSE  : 0.0069
RMSE : 0.0833
